In [1]:
import pandas as pd
import numpy as np
from sklearn import neighbors
import matplotlib.pyplot as ptl
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH


In [2]:
trip = pd.read_csv('data/trip_train.csv')

In [3]:
trip.sample(2)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
283211,908670,260,8/27/2015 17:35,Broadway St at Battery St,82,8/27/2015 17:39,Mechanics Plaza (Market at Battery),75,375,Subscriber,94114
75005,641254,612,2/12/2015 9:38,San Francisco Caltrain 2 (330 Townsend),69,2/12/2015 9:48,5th at Howard,57,320,Subscriber,94403


weather = pd.read_csv('data/weather.csv')

weather.columns

In [4]:
def splitDateAndTime(dateAndTime):
    splitLine = dateAndTime.split(' ')
    return str(splitDate(splitLine[0]))  + splitTime(splitLine[1])

def splitDate(date):
    lineSplit = date.split('/')
    return str(lineSplit[2]) + str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1] if int(lineSplit[1]) >= 10 else '0' + str(lineSplit[1]) )

def splitTime(time):
    lineSplit = time.split(':')
    return str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1])


weather.sample(2)

In [5]:
modifTrip = trip[['start_date','duration','start_station_name','start_station_id','subscription_type']]

· Agregar columna para dia de la semana

· Agregar columna para hora del dia

· Agregar columna para si es festividad o no

· Agregar columna para el mes

In [6]:
modifTrip.loc[:,'time'] = modifTrip['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
modifTrip.loc[:,'start_date'] = modifTrip['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))

In [8]:
modifTrip.sample(1)

,start_date,duration,start_station_name,start_station_id,subscription_type,time
31115,2015-03-27 08:03:00,822,San Francisco Caltrain (Townsend at 4th),70,Subscriber,803


In [9]:
modifTrip.loc[:,'day_of_week'] = modifTrip['start_date'].apply(lambda x : x.dayofweek )

In [10]:
modifTrip.loc[:,'month'] = modifTrip['start_date'].apply(lambda x : x.month )

In [11]:
modifTrip.loc[:,'year'] = modifTrip['start_date'].apply(lambda x : x.year )

In [12]:
modifTrip.loc[:,'dayofyear'] = modifTrip['start_date'].apply(lambda x : x.dayofyear )

In [13]:
modifTrip.loc[:,'start_date'] = modifTrip['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [14]:
modifTrip.sample(1)

,start_date,duration,start_station_name,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear
342906,2014-10-16,434,Powell Street BART,39,Subscriber,723,3,10,2014,289


In [15]:
cal = calendarUSFH()
holidays = cal.holidays(return_name=True,start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))
holiday_festive_day = pd.DataFrame(holidays,columns=['holiday']).reset_index()
holiday_festive_day.rename(columns={'index':'start_date'},inplace=True)


In [16]:
holiday_festive_day.sample(1)

,start_date,holiday
9,2013-12-25,Christmas


In [17]:
holiday_festive_day.loc[:,'start_date'] = holiday_festive_day['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [18]:
tripsModi = pd.merge(holiday_festive_day,modifTrip,on=['start_date'],how='right')

In [19]:
tripsModi.loc[:,'is_holiday'] = tripsModi.holiday.isnull()

In [20]:
tripsModi.loc[:,'is_holiday'] = tripsModi['is_holiday'].apply(lambda x : not x )
tripsModi.sample(2)

,start_date,holiday,duration,start_station_name,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
33507,2013-11-27,NaN,927,Temporary Transbay Terminal (Howard at Beale),55,Subscriber,826,2,11,2013,331,False
301024,2015-07-31,NaN,181,Temporary Transbay Terminal (Howard at Beale),55,Subscriber,1839,4,7,2015,212,False


In [21]:
tripsModi.shape

(549961, 12)

In [22]:
target = tripsModi[['duration']]

In [23]:
train = tripsModi.drop(labels=['start_date','duration','start_station_name','holiday'],axis=1)

In [24]:
train.loc[:,'subscription_type'] = train['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )

In [25]:
train.sample()

,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
238659,68,0,1800,3,7,2015,211,False


In [26]:
print help(neighbors.KNeighborsRegressor)

Help on class KNeighborsRegressor in module sklearn.neighbors.regression:

class KNeighborsRegressor(sklearn.neighbors.base.NeighborsBase, sklearn.neighbors.base.KNeighborsMixin, sklearn.neighbors.base.SupervisedFloatMixin, sklearn.base.RegressorMixin)
 |  Regression based on k-nearest neighbors.
 |  
 |  The target is predicted by local interpolation of the targets
 |  associated of the nearest neighbors in the training set.
 |  
 |  Read more in the :ref:`User Guide <regression>`.
 |  
 |  Parameters
 |  ----------
 |  n_neighbors : int, optional (default = 5)
 |      Number of neighbors to use by default for :meth:`k_neighbors` queries.
 |  
 |  weights : str or callable
 |      weight function used in prediction.  Possible values:
 |  
 |      - 'uniform' : uniform weights.  All points in each neighborhood
 |        are weighted equally.
 |      - 'distance' : weight points by the inverse of their distance.
 |        in this case, closer neighbors of a query point will have a
 |   

In [27]:
knn = neighbors.KNeighborsRegressor(n_neighbors=9,n_jobs=-1,p=5 )

In [28]:
knn.fit(train,target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=9, p=5,
          weights='uniform')

In [29]:
test = tripsModi.sample(5000)

In [30]:
test_target = test['duration']

In [31]:
test = test.drop(labels=['start_date','duration','start_station_name','holiday'],axis=1)

In [32]:
test.loc[:,'subscription_type'] = test['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )

In [33]:
test.sample()

,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
438468,62,0,857,0,1,2015,5,False


In [34]:
test_prediction = knn.predict(test)

In [35]:
predictionDF = pd.DataFrame(test_prediction,columns={"prediction"})

In [36]:
predictionDF.loc[:,'spectate'] = test_target.values

In [37]:
predictionDF.sample(5)

,prediction,spectate
3650,505.888889,544
211,997.333333,363
3936,387.111111,290
3335,443.000000,233
3854,457.222222,498


In [38]:
def predictError(predict,spectate):
    if (spectate > predict) : return  ((spectate-predict)/spectate)*100
    else  :  return ((predict-spectate)/predict )*100

In [39]:
predictionDF['errorValue'] = predictionDF.apply(axis=1,func=lambda x : predictError(x[0],x[1]))

In [40]:
predictionDF.sample(5)

,prediction,spectate,errorValue
565,602.333333,1272,52.646751
2750,659.111111,1185,44.378809
3349,1544.444444,4085,62.192302
4196,479.555556,577,16.888119
1429,719.333333,756,4.850088


In [41]:
print 'El error promedio que comete KNN al predecir es de', float(predictionDF['errorValue'].mean()),'%'

El error promedio que comete KNN al predecir es de 36.7903467029 %


In [49]:
print "El algoritmo KNN predice un ",\
float(predictionDF[predictionDF['errorValue'] < 15].shape[0]) / float(predictionDF.shape[0]) *100 ,\
"% de los valores, con un error menor al 15%"


El algoritmo KNN predice un  22.68 % de los valores, con un error menor al 15%
